In [1]:
import torch
import onnx
from brevitas.export import export_qonnx

from finn.util.visualization import showInNetron

from finn.util.test import get_test_model_trained
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_datatypes import InferDataTypes

import numpy as np
import qonnx.core.onnx_exec as oxe

import cv2

### Export to QONNX during training and test with DFire MINI

In [2]:
ori_onnx = 'BED_classifier__best_mean_F1__BIPOLAR_Out____NoTensorNorm__QONNX.onnx'
clean_onnx = '01_clean.onnx'

In [3]:
qonnx_cleanup(ori_onnx, out_file=clean_onnx)

In [4]:
showInNetron(ori_onnx)

Serving 'BED_classifier__best_mean_F1__BIPOLAR_Out____NoTensorNorm__QONNX.onnx' at http://0.0.0.0:8083


In [5]:
showInNetron(clean_onnx)

Stopping http://0.0.0.0:8083
Serving '01_clean.onnx' at http://0.0.0.0:8083


In [6]:
finn_tidy_onnx =  '02_finn_tidy.onnx'

In [7]:
model = ModelWrapper(clean_onnx)
model = model.transform(ConvertQONNXtoFINN())
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())
model.save(finn_tidy_onnx)

In [8]:
showInNetron(finn_tidy_onnx)

Stopping http://0.0.0.0:8083
Serving '02_finn_tidy.onnx' at http://0.0.0.0:8083


# Compare Outputs

In [9]:
dummy_in = np.random.randint(low = 0, high = 255+1, size = (1, 3, 224, 224)) 
dummy_in = (dummy_in / 256.).astype(np.float32)

In [10]:
#ori_model = ModelWrapper(ori_onnx)
clean_model = ModelWrapper(clean_onnx)
finn_model = ModelWrapper(finn_tidy_onnx)

### Cleaned QONNX

In [11]:
input_dict = {"global_in": dummy_in}
output_dict = oxe.execute_onnx(clean_model, input_dict)
produced_clean_qonnx = output_dict[list(output_dict.keys())[0]]
produced_clean_qonnx

array([[ 1., -1.]], dtype=float32)

### FINN QONNX

In [12]:
input_dict = {"global_in": dummy_in}
output_dict = oxe.execute_onnx(finn_model, input_dict)
produced_finn_qonnx = output_dict[list(output_dict.keys())[0]]
produced_finn_qonnx

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_0_out0 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(


array([[ 1., -1.]], dtype=float32)

# Compare with Images

In [13]:
img_file = './WEB10495.jpg' # Smoke and Fire
#img_file = './WEB10980.jpg' # Only Smoke
#img_file = './WEB10031.jpg' # Empty
img = cv2.imread(img_file)
img = cv2.imread(img_file)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
img = cv2.resize(img, (224,224), interpolation = cv2.INTER_LINEAR) 
img = img / 256.
img = np.expand_dims(img, axis=0)
img = np.transpose(img, (0, 3, 1, 2))
img = img.astype(np.float32)

In [14]:
print(f'Image shape: {img.shape}')
print(f'Data type: {img.dtype}')

Image shape: (1, 3, 224, 224)
Data type: float32


### Cleaned QONNX

In [15]:
input_dict = {"global_in": img}
output_dict = oxe.execute_onnx(clean_model, input_dict)
produced_clean_qonnx = output_dict[list(output_dict.keys())[0]]
produced_clean_qonnx

array([[1., 1.]], dtype=float32)

### FINN QONNX

In [16]:
input_dict = {"global_in": img}
output_dict = oxe.execute_onnx(finn_model, input_dict)
produced_finn_qonnx = output_dict[list(output_dict.keys())[0]]
produced_finn_qonnx

array([[1., 1.]], dtype=float32)

# Add Preprocessing Node: /255. -> ToTensor, FINN function

In [17]:
from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType

In [20]:
model = ModelWrapper(finn_tidy_onnx)
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = "./preproc_BED_input.onnx"
export_qonnx(totensor_pyt, torch.randn(ishape), chkpt_preproc_name)
qonnx_cleanup(chkpt_preproc_name, out_file=chkpt_preproc_name)
pre_model = ModelWrapper(chkpt_preproc_name)
pre_model = pre_model.transform(ConvertQONNXtoFINN())

# join preprocessing and core model
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all BNN-PYNQ models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/merge_onnx_models.py:70: UserWarning: [MergeONNXModels] opsets for models to merge differ: 14 vs 17, output model will use opset 17
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


### Tidy up again and save

In [21]:
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

finn_tidy_preprocess = '03_finn_prepro.onnx'
model.save(finn_tidy_preprocess)

In [22]:
showInNetron(finn_tidy_preprocess)

Stopping http://0.0.0.0:8083
Serving '03_finn_prepro.onnx' at http://0.0.0.0:8083


# Streamline the Model

In [23]:
from finn.util.visualization import showSrc
from finn.transformation.streamline import Streamline
showSrc(Streamline)

class Streamline(Transformation):
    """Apply the streamlining transform, see arXiv:1709.04060."""

    def apply(self, model):
        streamline_transformations = [
            ConvertSubToAdd(),
            ConvertDivToMul(),
            BatchNormToAffine(),
            ConvertSignToThres(),
            MoveMulPastMaxPool(),
            MoveScalarLinearPastInvariants(),
            AbsorbSignBiasIntoMultiThreshold(),
            MoveAddPastMul(),
            MoveScalarAddPastMatMul(),
            MoveAddPastConv(),
            MoveScalarMulPastMatMul(),
            MoveScalarMulPastConv(),
            MoveAddPastMul(),
            CollapseRepeatedAdd(),
            CollapseRepeatedMul(),
            MoveMulPastMaxPool(),
            AbsorbAddIntoMultiThreshold(),
            FactorOutMulSignMagnitude(),
            AbsorbMulIntoMultiThreshold(),
            Absorb1BitMulIntoMatMul(),
            Absorb1BitMulIntoConv(),
            RoundAndClipThresholds(),
        ]
        for tr

In [25]:
from finn.transformation.streamline import Streamline
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveScalarLinearPastInvariants
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.general import RemoveUnusedTensors

import finn.transformation.streamline.reorder as reorder
from qonnx.transformation.change_datalayout import ChangeDataLayoutQuantAvgPool2d

In [26]:
model = ModelWrapper(finn_tidy_preprocess)
model = model.transform(MoveScalarLinearPastInvariants())
model = model.transform(Streamline())
model = model.transform(LowerConvsToMatMul())
model = model.transform(MakeMaxPoolNHWC())
model = model.transform(ChangeDataLayoutQuantAvgPool2d()) # Wortel
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(Streamline())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

In [27]:
finn_streamline = './04_finn_streamline.onnx'
model.save(finn_streamline)

In [28]:
showInNetron(finn_streamline)

Stopping http://0.0.0.0:8083
Serving './04_finn_streamline.onnx' at http://0.0.0.0:8083


# Partitioning, Conversion to HW Layers and Folding

In [29]:
from finn.util.basic import pynq_part_map
# change this if you have a different PYNQ board, see list above
pynq_board = "Pynq-Z1"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

In [30]:
print(fpga_part)

xc7z020clg400-1


In [31]:
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers
from qonnx.custom_op.registry import getCustomOp
from qonnx.transformation.infer_data_layouts import InferDataLayouts

### HW Layers

https://github.com/Xilinx/finn/blob/main/src/finn/transformation/fpgadataflow/convert_to_hw_layers.py

Added InferGlobalAccPoolLayer to transform GlobalAvgPooling

In [32]:
model = ModelWrapper(finn_streamline)

# MVAUs
#model = model.transform(to_hw.InferBinaryMatrixVectorActivation()) # -> Wort: no binary mtx mult in my model
#model = model.transform(to_hw.InferVectorVectorActivation()) # -> Wort: esto es para conv depthwise
model = model.transform(to_hw.InferQuantizedMatrixVectorActivation())

# input quantization (if any) to standalone thresholding
model = model.transform(to_hw.InferThresholdingLayer())
model = model.transform(to_hw.InferConvInpGen())
model = model.transform(to_hw.InferStreamingMaxPool())
model = model.transform(to_hw.InferPool())

# get rid of Reshape(-1, 1) operation between hw nodes
model = model.transform(RemoveCNVtoFCFlatten())

# get rid of Tranpose -> Tranpose identity seq
model = model.transform(absorb.AbsorbConsecutiveTransposes())

# infer tensor data layouts
model = model.transform(InferDataLayouts())

In [33]:
finn_hw_layers = './05_finn_hw_layers.onnx'
model.save(finn_hw_layers)

In [34]:
showInNetron(finn_hw_layers)

Stopping http://0.0.0.0:8083
Serving './05_finn_hw_layers.onnx' at http://0.0.0.0:8083


In [29]:
parent_model = model.transform(CreateDataflowPartition())
parent_model.save('./BED_classifier__best_mean_F1_finn_parent__perChannel_fxPoint__QONNX.onnx')
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_model_filename = sdp_node.get_nodeattr("model")
# save the dataflow partition with a different name for easier access
# and specialize the layers to HLS variants
dataflow_model = ModelWrapper(dataflow_model_filename)
dataflow_model = dataflow_model.transform(SpecializeLayers(fpga_part))
dataflow_model.save('./BED_classifier__best_mean_F1_finn_dataflow__perChannel_fxPoint__QONNX.onnx')

AssertionError: cycle-free graph violated: partition depends on itself